In [1]:
import pandas as pd
import h5py
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split,  WeightedRandomSampler
import os
import torch
import albumentations as A
from torchvision.models import efficientnet_b7, EfficientNet_B7_Weights
import torch.nn as nn
import torch.optim as optim
from torchmetrics import Accuracy
from torchmetrics.classification import F1Score, Precision, Recall
import pytorch_lightning as pl
from tqdm import tqdm
from lightning.pytorch.loggers import TensorBoardLogger
from torch.cuda.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split
import optuna
from torchmetrics import Metric
from sklearn.metrics import roc_curve, auc, roc_auc_score
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
torch.cuda.is_available()

True

## Dataset/DataModule

In [3]:
class ISIC_Dataset(Dataset):
    def __init__(self, data_path, dataframe, transform=None):
        self.data_path = data_path
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        
        img_name = os.path.join(self.data_path, self.dataframe['isic_id'].iloc[idx] + '.jpg')
        
        image = (np.array(Image.open(img_name).convert('RGB')) / 255)
        target = self.dataframe['target'].iloc[idx]
        
        if self.transform:
            image = self.transform(image=image)['image'].transpose(2, 0, 1)
            
        return torch.tensor(image, dtype=torch.float32), torch.tensor(target, dtype=torch.long)


# class ISICDataModule(pl.LightningDataModule):
#     def __init__(self, data_path, dataframe, batch_size=32, transform=None):
#         super().__init__()
#         self.data_path = data_path
#         self.dataframe = dataframe
#         self.batch_size = batch_size
#         self.transform = transform
#         self.train_df, self.val_df = self.get_train_test_dfs(test_size = 0.3)
#         self.val_dataset = ISIC_Dataset(self.data_path, self.val_df, transform=self.transform)

#     def setup(self, stage=None):
        
#         self.train_dataset = ISIC_Dataset(self.data_path, self.train_df, transform=self.transform)
#         self.val_dataset = ISIC_Dataset(self.data_path, self.val_df, transform=self.transform)

#     def get_train_test_dfs(self, test_size):
#         X_df = self.dataframe.drop('target', axis = 'columns')
#         y_df = self.dataframe['target']
#         #y = self.dataframe['target'].values
#         # Perform a stratified train-test split
#         X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=test_size, stratify=y_df)
        
#         # Combine X and y into single DataFrames
#         train_df = X_train.copy()
#         train_df['target'] = y_train.values
        
#         val_df = X_test.copy()
#         val_df['target'] = y_test.values

#         return train_df, val_df
        
        

#     def get_sampler(self, df):
#         sample_weights = np.zeros(shape = (len(df)))
#         vc = df['target'].value_counts()
#         sample_weights[df['target'] == 1] = (1 / vc[1])
#         sample_weights[df['target'] == 0] = (1 / vc[0])
#         sampler = WeightedRandomSampler(weights = sample_weights, num_samples=len(sample_weights), replacement=False)
#         return sampler
        
        
#     def train_dataloader(self):
#         sampler = self.get_sampler(self.train_df)
#         return DataLoader(self.train_dataset, batch_size=self.batch_size,sampler = sampler, pin_memory = True, num_workers = 16)

#     def val_dataloader(self):
#         sampler = self.get_sampler(self.val_df)
#         return DataLoader(self.val_dataset, batch_size=self.batch_size,sampler = sampler, pin_memory = True, num_workers = 16)


## Model/LightningModule

In [20]:
import pdb
class SaveBestModel(pl.Callback):
    def __init__(self, filepath = './best_model/', monitor='val_loss', save_best_only=True):
        
        
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.best = float('inf')

    def on_validation_end(self, trainer, pl_module):
        current = trainer.callback_metrics[self.monitor].item()
        
        if self.save_best_only:
            if current < self.best:
                self.best = current
                torch.save(pl_module.state_dict(), self.filepath)
                print(f"Validation {self.monitor} improved to {current:.4f}, saving model to {self.filepath}")


class pAUC(Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.total_preds = []
        self.labels = []

    
    def update(self, preds, target):
        # Convert predictions to class labels
        preds = torch.argmax(preds, dim=1)
        self.total_preds.extend(preds)
        self.labels.extend(target)
        
    def compute(self, min_tpr = 0.8):
        preds = torch.tensor(self.total_preds)
        lbls = torch.tensor(self.labels)
        
        v_gt = abs(np.array(lbls.cpu())-1)
        v_pred = -1.0*np.array(preds.cpu())
    
        max_fpr = abs(1-min_tpr)
    
        # using sklearn.metric functions: (1) roc_curve and (2) auc
        fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
        if max_fpr is None or max_fpr == 1:
            return auc(fpr, tpr)
        if max_fpr <= 0 or max_fpr > 1:
            raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
            
        # Add a single point at max_fpr by linear interpolation
        stop = np.searchsorted(fpr, max_fpr, "right")
        x_interp = [fpr[stop - 1], fpr[stop]]
        y_interp = [tpr[stop - 1], tpr[stop]]
        tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
        fpr = np.append(fpr[:stop], max_fpr)
        partial_auc = auc(fpr, tpr)
    
        #     # Equivalent code that uses sklearn's roc_auc_score
        #     v_gt = abs(np.asarray(solution.values)-1)
        #     v_pred = np.array([1.0 - x for x in submission.values])
        #     max_fpr = abs(1-min_tpr)
        #     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
        #     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
        #     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
        #     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
        
        return torch.tensor(partial_auc)




class EfficientNetBinaryClassifier(nn.Module):
    def __init__(self, efficientnet, num_features):
        super(EfficientNetBinaryClassifier, self).__init__()

        self.efficientnet = efficientnet
        self.dropout = nn.Dropout()
        self.num_features = num_features
        
        self.classifier = nn.Linear(num_features, 1)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.dropout(x)
        x = self.classifier(x.view(-1, self.num_features))
        return x

class EfficientNetBinaryClassifierLightning(pl.LightningModule):
    def __init__(self, efficientnet, val_loader, num_features, optim, lr=1e-3):
        super(EfficientNetBinaryClassifierLightning, self).__init__()
        self.model = EfficientNetBinaryClassifier(efficientnet, num_features)
        self.criterion = nn.BCEWithLogitsLoss()
        self.lr = lr
        self.train_accuracy = Accuracy('binary')
        self.val_accuracy = Accuracy('binary')
        num_classes = 2
        self.f1 = F1Score(num_classes=num_classes, average='weighted', task='binary')
        self.precision = Precision(num_classes=num_classes, average='weighted', task='binary')
        self.recall = Recall(num_classes=num_classes, average='weighted', task='binary')
        self.p_auc = pAUC()
        self.validation_dataloader = val_loader
        self.optim_choice = optim
        

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.view(-1, 1).float()
        #pdb.set_trace()
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = self.train_accuracy(preds, y.int())
        labels = y.int()
        f1 = self.f1(preds, labels)
        self.precision(preds, labels)
        self.recall(preds, labels)
        self.log('train_loss', loss, prog_bar=True, on_epoch = True, on_step = True)
        
        # self.eval()  # Switch to evaluation mode
        # # with torch.no_grad():  # Disable gradient computation
        # #     for val_batch_idx, val_batch in enumerate(self.validation_dataloader):
        # #         self.validation_step(val_batch, val_batch_idx)
        # # self.train()  # Switch back to training mode
        # x,y = self.validation_step()
        return loss

    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = y.view(-1, 1).float()
        
        logits = self(x)
        #pdb.set_trace()
        loss = self.criterion(logits, y)
        preds = torch.sigmoid(logits) > 0.5
        acc = self.val_accuracy(preds, y.int())
        labels = y.int()
        self.f1(preds, labels)
        self.precision(preds, labels)
        self.recall(preds, labels)
        self.p_auc(preds, labels)
        self.log('val_loss', loss, prog_bar=True, on_epoch = True)
        self.log('val_acc', acc, prog_bar=True, on_epoch = True)
        self.log('val_f1', self.f1, prog_bar=True, on_epoch = True)
        self.log('val_precision', self.precision, prog_bar=True, on_epoch = True)
        self.log('val_recall', self.recall, prog_bar=True, on_epoch = True)
        self.log('val_pAUC', self.p_auc, prog_bar=True, on_epoch=True)
   
        

    def configure_optimizers(self):
        if self.optim_choice == 'Adam':
            return torch.optim.Adam(self.parameters(), lr=self.lr)
        else:
            return optim.SGD(model.parameters(), lr=self.lr, momentum=0.9, nesterov=True)


In [21]:
def get_train_test_dfs(dataframe,test_size):
        X_df = dataframe.drop('target', axis = 'columns')
        y_df = dataframe['target']
        #y = self.dataframe['target'].values
        # Perform a stratified train-test split
        X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=test_size, stratify=y_df)
        
        # Combine X and y into single DataFrames
        train_df = X_train.copy()
        train_df['target'] = y_train.values
        
        val_df = X_test.copy()
        val_df['target'] = y_test.values

        return train_df, val_df

def get_sampler(df):
        sample_weights = np.zeros(shape = (len(df)))
        vc = df['target'].value_counts()
        sample_weights[df['target'] == 1] = (1 / vc[1])
        sample_weights[df['target'] == 0] = (1 / vc[0])
        sampler = WeightedRandomSampler(weights = sample_weights, num_samples=len(sample_weights), replacement=False)
        return sampler
    
        

In [22]:
df_main = pd.read_csv('train-metadata.csv')
df_add = pd.read_csv('metadata.csv')
df_add = df_add[~df_add['benign_malignant'].isna()]
df_add['benign_malignant'].value_counts()
df_add = df_add[df_add['benign_malignant']!= 'indeterminate']
df_add['target'] = df_add['benign_malignant'].apply(func = lambda x : 1 if 'malignant' in x else 0)
common_cols = ['isic_id', 'target']
data = pd.concat([df_add[common_cols], df_main[common_cols]], axis = 'rows')
data_mini = pd.concat((data[data['target'] == 1], data[data['target'] == 0].iloc[:1000])).reset_index().sample(frac = 1)
data_test = pd.read_csv('test-metadata.csv')
len(data)

/tmp/ipykernel_14004/1879230498.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_main = pd.read_csv('train-metadata.csv')
/tmp/ipykernel_14004/1879230498.py:2: DtypeWarning: Columns (8,13,16,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_add = pd.read_csv('metadata.csv')


474497

In [23]:
data_path = './train-image/image/'
transform = A.Compose([
    A.Resize(100, 100),                    # Resize the image to 224x224
    A.Rotate(limit=30, p=0.5)              # Rotate the image by ±30 degrees with 50% probability
])
batch_size = 32
# Initialize the EfficientNet model (Assuming using EfficientNet-B0 for example)
efficientnet = efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1)
num_features = efficientnet.classifier[1].in_features  # Assuming '_fc' is the final fully connected layer
train_df, val_df = get_train_test_dfs(dataframe=data_mini, test_size = 0.3)
        
# Replace the final layer with an identity layer
efficientnet.classifier = nn.Identity()
train_dataset = ISIC_Dataset(data_path, train_df, transform=transform)
val_dataset = ISIC_Dataset(data_path, val_df, transform=transform)
train_sampler = get_sampler(train_df)
val_sampler = get_sampler(val_df)
# Initialize DataModule and Model
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler = train_sampler, pin_memory = True, num_workers = 16)
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler = val_sampler, pin_memory = True, num_workers = 16)
#checkpoint_callback = SaveBestModel(monitor='val_loss', save_best_only=True)
#data_module = ISICDataModule(data_path, dataframe, batch_size=32, transform=transform)
model = EfficientNetBinaryClassifierLightning(efficientnet, val_loader, num_features, optim = 'Adam')
logger = TensorBoardLogger("tb_logs", name="my_model")
# Initialize Trainer
trainer = pl.Trainer(logger = logger, max_epochs = 20, accelerator = 'auto')

# Train and validate the model
trainer.fit(model, train_loader, val_loader)


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                         | Params | Mode 
------------------------------------------------------------------------
0 | model          | EfficientNetBinaryClassifier | 63.8 M | train
1 | criterion      | BCEWithLogitsLoss            | 0      | train
2 | train_accuracy | BinaryAccuracy               | 0      | train
3 | val_accuracy   | BinaryAccuracy               | 0      | train
4 | f1             | BinaryF1Score                | 0      | train
5 | precision  

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
class pAUC(Metric):
    def __init__(self, compute_on_step=False, dist_sync_on_step=False, process_group=None, dist_sync_on_epoch=False):
        super().__init__(compute_on_step=compute_on_step, dist_sync_on_step=dist_sync_on_step,
                         process_group=process_group, dist_sync_on_epoch=dist_sync_on_epoch)

        self.add_state("correct", default_size=(1,), dist_reduce="sum") 
        self.add_state("total", default_size=(1,), dist_reduce="sum") 
        self.total_preds = []
        self.labels = [from sklearn.metrics import roc_curve, auc, roc_auc_score
]
    
    def update(self, preds, target):
        # Convert predictions to class labels
        preds = torch.argmax(preds, dim=1)
        self.correct += (preds == target).sum()
        self.total += target.numel()

    def compute(self):
        v_gt = abs(np.asarray(solution.values)-1)
    
        # flip the submissions to their compliments
        v_pred = -1.0*np.asarray(submission.values)
    
        max_fpr = abs(1-min_tpr)
    
        # using sklearn.metric functions: (1) roc_curve and (2) auc
        fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
        if max_fpr is None or max_fpr == 1:
            return auc(fpr, tpr)
        if max_fpr <= 0 or max_fpr > 1:
            raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
            
        # Add a single point at max_fpr by linear interpolation
        stop = np.searchsorted(fpr, max_fpr, "right")
        x_interp = [fpr[stop - 1], fpr[stop]]
        y_interp = [tpr[stop - 1], tpr[stop]]
        tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
        fpr = np.append(fpr[:stop], max_fpr)
        partial_auc = auc(fpr, tpr)
    
        #     # Equivalent code that uses sklearn's roc_auc_score
        #     v_gt = abs(np.asarray(solution.values)-1)
        #     v_pred = np.array([1.0 - x for x in submission.values])
        #     max_fpr = abs(1-min_tpr)
        #     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
        #     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
        #     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
        #     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
        
        return(partial_auc)
        







In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    # We optimize the number of layers, hidden units in each layer and dropouts.
    lr = trial.suggest_float("lr", 1e-5, 1e-1)
    optim_choice = trial.suggest_categorical("optim_choice", ['Adam', 'SGD'])
    
        
    
    data_path = './train-image/image/'
    transform = A.Compose([
        A.Resize(100, 100),                   
        A.Rotate(limit=30, p=0.5)             
    ])
    batch_size = 32
    # Initialize the EfficientNet model (Assuming using EfficientNet-B0 for example)
    efficientnet = efficientnet_b7(weights=EfficientNet_B7_Weights.IMAGENET1K_V1)
    num_features = efficientnet.classifier[1].in_features  # Assuming '_fc' is the final fully connected layer
    train_df, val_df = get_train_test_dfs(dataframe=data_mini, test_size = 0.3)
            
    # Replace the final layer with an identity layer
    efficientnet.classifier = nn.Identity()
    train_dataset = ISIC_Dataset(data_path, train_df, transform=transform)
    val_dataset = ISIC_Dataset(data_path, val_df, transform=transform)
    train_sampler = get_sampler(train_df)
    val_sampler = get_sampler(val_df)
    # Initialize DataModule and Model
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler = train_sampler, pin_memory = True, num_workers = 16)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler = val_sampler, pin_memory = True, num_workers = 16)
    
    #data_module = ISICDataModule(data_path, dataframe, batch_size=32, transform=transform)
    model = EfficientNetBinaryClassifierLightning(efficientnet, val_loader, num_features, optim_choice, lr)
    logger = TensorBoardLogger("tb_logs", name="my_model")
    # Initialize Trainer
    trainer = pl.Trainer(logger = logger, max_epochs = 1, accelerator = 'auto', callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_pAUC")])
    
    # Train and validate the model
    trainer.fit(model, train_loader, val_loader)

    return trainer.callback_metrics["val_pAUC"].item()


In [18]:
data['target'].value_counts()

target
0    464780
1      9717
Name: count, dtype: int64

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="PyTorch Lightning example.")
    parser.add_argument(
        "--pruning",
        "-p",
        action="store_true",
        help="Activate the pruning feature. `MedianPruner` stops unpromising "
        "trials at the early stages of training.",
    )
    args = parser.parse_args()

    pruner = optuna.pruners.MedianPruner() if args.pruning else optuna.pruners.NopPruner()

    study = optuna.create_study(direction="maximize", pruner=pruner)
    study.optimize(objective, n_trials=100, timeout=600)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))